In [ ]:
# MLP - with using paper Bengio et al.2003

In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt # for making figures
%matplotlib inline

In [3]:
# read in all the words
words = open('cleaned_names.txt', 'r').read().splitlines()
words[:8]

['aaban', 'aabid', 'aabidah', 'aabir', 'aabriella', 'aada', 'aadam', 'aadarsh']

In [4]:
len(words)

29681

In [5]:
# build the vocabulary of characters and mappings to/from integers
chars = sorted(set(''.join(words)))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [10]:
# build the dataset

block_size = 3 # context length: how many characters to use to predict the next one?
X, Y = [], []
for w in words[:5]:

  print(w)
  context = [0] * block_size
  for ch in w + '.':
    ix = stoi[ch]
    X.append(context)
    Y.append(ix)
    print(''.join([itos[i] for i in context]), '---->', itos[ix])
    context = context[1:] + [ix] # crop and append

X = torch.tensor(X)
Y = torch.tensor(Y)

aaban
... ----> a
..a ----> a
.aa ----> b
aab ----> a
aba ----> n
ban ----> .
aabid
... ----> a
..a ----> a
.aa ----> b
aab ----> i
abi ----> d
bid ----> .
aabidah
... ----> a
..a ----> a
.aa ----> b
aab ----> i
abi ----> d
bid ----> a
ida ----> h
dah ----> .
aabir
... ----> a
..a ----> a
.aa ----> b
aab ----> i
abi ----> r
bir ----> .
aabriella
... ----> a
..a ----> a
.aa ----> b
aab ----> r
abr ----> i
bri ----> e
rie ----> l
iel ----> l
ell ----> a
lla ----> .
